In [47]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [48]:
# Cargamos el dataframe
df = pd.read_csv('../data/train.csv', index_col='id', parse_dates=['fecha'])

In [49]:
df.shape

(240000, 22)

In [50]:
df.head(2)

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00


In [51]:
# Veo la cantidad de elementos nulos de cada columna
display(df.isnull().sum())

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

# Optimizando los tipos de datos
### Integers
<span style="font-weight:600;">Lleno los elementos nulos de tipo tipo float con el valor -1, para que sea un valor valido para la conversion a int, pero al no coincidir con los valores reales de las columnas luego los paso a nan</span>

In [52]:
df[['antiguedad','habitaciones','garages','banos','idzona']] = df[['antiguedad','habitaciones','garages','banos','idzona']].fillna(-1)

<span style="font-weight:600;">Cambio el tipo de todas las columnas que son float y que en verdad se representan en int</span>

In [53]:
df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']] = pd.DataFrame(df, columns=['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio'], dtype=int)

<span style="font-weight:600;">Paso del tipo de dato int64 que se coloco en el paso anterior , y lo downcasteo a el minimo posible para cada columna (int32, int16, etc) , y recupero los datos nulos</span>

In [54]:
df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']] = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'idzona', 'precio']].apply(pd.to_numeric, downcast='integer').replace(-1, np.nan)

### Floats
<span style="font-weight:600;">Lleno los elementos nulos de tipo tipo float con el valor -1, para que sea un valor valido, pero que sea facil de identificar como valor vacio, al no coincidir con los valores reales de las columnas luego los paso a nan</span>

In [55]:
df[['metroscubiertos','metrostotales']] = df[['metroscubiertos','metrostotales']].fillna(-1)

<span style="font-weight:600;">Downcasteo a el minimo posible para cada columna (float32, float16, etc) y recupero los nulos</span>

In [56]:
df[['metroscubiertos', 'metrostotales']] = df[['metroscubiertos', 'metrostotales']].apply(pd.to_numeric,downcast='float').replace(-1, np.nan)

### Booleans y Category
<span style="font-weight:600;">Cambio el tipo de dato de todas las columnas booleanas a int 1/0 y la columna que se representa como category</span>

In [57]:
df[['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']] = pd.DataFrame(df, columns=['gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos'], dtype=int).apply(pd.to_numeric, downcast='integer')
df = df.astype({'tipodepropiedad': 'category'})

## Resultado
### Tipo de datos final para cada columna

In [58]:
df.dtypes

titulo                                object
descripcion                           object
tipodepropiedad                     category
direccion                             object
ciudad                                object
provincia                             object
antiguedad                           float64
habitaciones                         float64
garages                              float64
banos                                float64
metroscubiertos                      float32
metrostotales                        float32
idzona                               float64
lat                                  float64
lng                                  float64
fecha                         datetime64[ns]
gimnasio                                int8
usosmultiples                           int8
piscina                                 int8
escuelascercanas                        int8
centroscomercialescercanos              int8
precio                                 int32
dtype: obj

### Cantidad de nulos por columna

In [59]:
display(df.isnull().sum())

titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

# Analisis de datos

In [60]:
# Retiro las columnas de "lat" y "lng" dado que en el 50% de los datos estan vacios
# df = df.drop(columns=['lat', 'lng'])

### Ordeno las filas por Fecha , id y precio

In [122]:
df = df.sort_values(by=["fecha","id","precio"])

### Conteo de la cantidad de publicaciones de venta por año

In [135]:
anios = pd.DataFrame(df[['fecha']], columns=['fecha'])
anios = anios["fecha"].groupby(anios['fecha'].dt.year).agg({'count'})
anios

,count
fecha,
2012,23534
2013,30386
2014,40572
2015,51470
2016,94038


### Conteo de la cantidad de publicaciones de venta por provincias

In [136]:
df.head()

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,,,
814,casa en venta en bosques de la herradura,bonita casa en tres niveles construida para pe...,Casa,2Âª CERRADA DE BOSQUE DE JIQUILPAN,Huixquilucan,Edo. de México,23,4,2,-1,-1.00,400.00,55570,nan,nan,2012-01-01,False,False,False,True,False,4500000
4369,casa en venta en guadalajara,excelente casa en muy buena ubicacion............,Casa,Isla Española 2603,Guadalajara,Jalisco,40,7,1,2,119.00,119.00,-1,20.64,-103.38,2012-01-01,False,False,False,True,True,1900000
7128,casa en condominio en venta en fraccionamiento...,"magnifica casa en condominio privado, con coch...",Casa en condominio,Carr Antiguo Aeropuerto,Querétaro,Querétaro,4,3,-1,2,430.00,-1.00,84007,nan,nan,2012-01-01,False,False,False,True,False,5500000
9702,casa en condominio en venta en san andrés cholula,hermosa y vanguardista casa por forjadores jun...,Casa en condominio,Melchor Ocampo 12,San Andrés Cholula,Puebla,0,3,2,2,102.00,84.00,-1,nan,nan,2012-01-01,False,False,False,False,False,900000
23940,departamento en venta en interlomas,"hermoso departamento seminuevo, moderno y muy...",Apartamento,Jesus del Monte,Huixquilucan,Edo. de México,5,3,-1,3,301.00,-1.00,55584,nan,nan,2012-01-01,False,False,True,True,False,5700000


In [140]:
provincias = pd.DataFrame(df, columns=['provincia'])
provincias = provincias["provincia"].groupby(provincias['provincia']).agg({'count'})
provincias = provincias.sort_values(by=["count"])
provincias.head()

,count
provincia,
Zacatecas,94
Campeche,263
Oaxaca,711
Tlaxcala,839
Tabasco,994
